# Download 1/2 degree plots

In [1]:
import ee, geemap
print(ee.__version__)
print(geemap.__version__)

0.1.249
0.8.8


In [2]:
ee.Initialize()

### Get region data and sentinel-2 data from GEE

In [3]:
# List sub-regions using GAUL polygon data
br = (ee.FeatureCollection("FAO/GAUL/2015/level1")
      .filterMetadata('ADM0_NAME', 'equals', 'Brazil')
      .aggregate_array('ADM1_NAME')
     )

print(br.getInfo())

['Alagoas', 'Bahia', 'Ceara', 'Espirito Santo', 'Maranhao', 'Minas Gerais', 'Paraiba', 'Pernambuco', 'Piaui', 'Rio De Janeiro', 'Rio Grande Do Norte', 'Sao Paulo', 'Sergipe', 'Name Unknown', 'Name Unknown', 'Name Unknown', 'Acre', 'Amapa', 'Amazonas', 'Distrito Federal', 'Goias', 'Mato Grosso', 'Mato Grosso Do Sul', 'Para', 'Parana', 'Rio Grande Do Sul', 'Rondonia', 'Roraima', 'Santa Catarina', 'Tocantins', 'Name Unknown']


In [4]:
# Get a sub-region from above to test grid function later
br = (ee.FeatureCollection("FAO/GAUL/2015/level1")
      .filterMetadata('ADM0_NAME', 'equals', 'Brazil')
      .filterMetadata('ADM1_NAME', 'equals', 'Rondonia')
     )

## Create a grid

Not 100% perfect. You have to have a reasonably-sized roi drawn. If it's too big, it's hard to predict where the grid will be drawn. But hey, at least it gives me some squares.

In [24]:
# Method 2: geemap.show_youtube('N7rK2aV1R4c')
# Create a map and draw a square interactively
Map = geemap.Map()
Map.addLayer(br)
Map.center_object(br, zoom=5)

Map

Map(center=[-4.186394628415986, -64.69261254089228], controls=(WidgetControl(options=['position', 'transparent…

In [5]:
# Set name
my_roi = br

In [6]:
# Save feature
my_roi = ee.FeatureCollection(my_roi)

In [16]:
# Create grid
# https://developers.google.com/earth-engine/tutorials/community/drawing-tools

def make_grid(region, a_scale):
    """
    Creates a grid around a specified ROI.
    User inputs their reasonably small ROI.
    User inputs a scale where 100000 = 100km.
    """
    # Creates image with 2 bands ('longitude', 'latitude') in degrees
    lonLat = ee.Image.pixelLonLat()

    # Select bands, multiply times big number, and truncate
    lonGrid = (lonLat
               .select('latitude')
               .multiply(10000000)
               .toInt()
              )
    latGrid = (lonLat
              .select('longitude')
              .multiply(10000000)
              .toInt()
              )

    # Multiply lat and lon images and reduce to vectors
    grid = (lonGrid
            .multiply(latGrid)
            .reduceToVectors(
                geometry = region,
                scale = a_scale, # 100km-sized boxes needs 100,000
                geometryType = 'polygon')
           )
    
    return(grid)

In [17]:
# Make test grid and add to map
grid_55km = make_grid(br, 55000)

## Visualize selected grid tiles + images

In [18]:
# Access coordinates of grid squares
grid_dict = grid_55km.getInfo()

feats = grid_dict['features']
coord_list = []
for d in feats:
    geom = d['geometry']
    coords = geom['coordinates']
    coord_list.append(coords)

In [19]:
# Create a list of several ee.Geometry.Polygons
polys = []
for coord in coord_list:
    poly = ee.Geometry.Polygon(coord)
    polys.append(poly)

In [20]:
# Limit grid tiles to export
#idx = list(range(0,100))
idx = [64] #64 is test area in rondonia
polys = [ polys[i] for i in idx]

In [21]:
# Make the whole grid a feature collection for export purposes
grid = ee.FeatureCollection(polys)

In [22]:
# Visualize a polygon or two
Map = geemap.Map()

Map.addLayer(grid)
Map.addLayer(polys[0])
Map.center_object(polys[0], zoom=5)

Map

Map(center=[-9.634459022148878, -64.47657951767819], controls=(WidgetControl(options=['position', 'transparent…

## Export grid as shapfile

In [21]:
# Set all ee.Geometry.Polygon to ee.Feature
feat_list = []
for poly in polys:
    feat = ee.Feature(poly)
    feat_list.append(feat)
print(len(feat_list))

1


In [23]:
# Export entire grid as shapefile
geemap.ee_to_shp(grid, filename='/data/6ru/shapefiles/rondonia_0.shp')

Generating URL ...
Please wait ...
Data downloaded to /data/6ru/shapefiles/rondonia_0.shp


## Make sub-grid from Rondonia AOI

In [38]:
# Import grid
grid_shp = '/data/6ru/shapefiles/rondonia_0.shp'
grid_fc = geemap.shp_to_ee(grid_shp)

In [39]:
# Create grid
# https://developers.google.com/earth-engine/tutorials/community/drawing-tools

def make_grid(region, a_scale):
    """
    Creates a grid around a specified ROI.
    User inputs their reasonably small ROI.
    User inputs a scale where 100000 = 100km.
    """
    # Creates image with 2 bands ('longitude', 'latitude') in degrees
    lonLat = ee.Image.pixelLonLat()

    # Select bands, multiply times big number, and truncate
    lonGrid = (lonLat
               .select('latitude')
               .multiply(10000000)
               .toInt()
              )
    latGrid = (lonLat
              .select('longitude')
              .multiply(10000000)
              .toInt()
              )

    # Multiply lat and lon images and reduce to vectors
    grid = (lonGrid
            .multiply(latGrid)
            .reduceToVectors(
                geometry = region,
                scale = a_scale, # 100km-sized boxes needs 100,000
                geometryType = 'polygon')
           )
    
    return(grid)

In [40]:
# Make test grid and add to map
grid_55km = make_grid(grid_fc, 27500)

In [41]:
# Access coordinates of grid squares
grid_dict = grid_55km.getInfo()

feats = grid_dict['features']
coord_list = []
for d in feats:
    geom = d['geometry']
    coords = geom['coordinates']
    coord_list.append(coords)

In [42]:
# Create a list of several ee.Geometry.Polygons
polys = []
for coord in coord_list:
    poly = ee.Geometry.Polygon(coord)
    polys.append(poly)

In [43]:
# Make the whole grid a feature collection for export purposes
grid = ee.FeatureCollection(polys)

In [44]:
# Visualize a polygon or two
Map = geemap.Map()

Map.addLayer(polys[0])
Map.addLayer(grid)
Map.center_object(polys[0], zoom=5)

Map

Map(center=[-9.510919896800196, -64.35306116611251], controls=(WidgetControl(options=['position', 'transparent…

In [45]:
# Export entire grid as shapefile
geemap.ee_to_shp(grid, filename='/data/6ru/shapefiles/rondonia_0_subgrid.shp')

Generating URL ...
Please wait ...
Data downloaded to /data/6ru/shapefiles/rondonia_0_subgrid.shp
